# Rare Variant Association Testing Tutorial

This tutorial demonstrates how to perform rare variant association testing using burden tests and SKAT (Sequence Kernel Association Test). We will walk through the steps of preparing the single-cell data (the same as for [common variant eQTL analysis](./pseudobulk_eqtl.ipynb)), running burden tests, and combining p-values across annotations. Additionally, we will explore SKAT for association testing.

The tutorial exemplifies the rare variant associationg testing procedure for a single cell type (CD 8 NC).

To get the VEP output files used in this tutorial check the [variant annotation tutorial](./explore_annotations.ipynb)

## Objectives
- Learn how to prepare genotype and phenotype data for rare variant association testing.
- Understand how to apply burden tests with different weighting schemes.
- Combine p-values across annotations using the Cauchy combination test.
- Perform SKAT for rare variant association testing.

In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import pandas as pd

In [41]:
import gc
from pathlib import Path
import warnings

import anndata as ad
import scanpy as sc
import dask.array as da
import numpy as np
from tqdm.auto import tqdm

import cellink as cl
from cellink._core import DAnn, GAnn
from cellink.tl._rvat import run_burden_test, run_skat_test, beta_weighting
from cellink.utils import column_normalize, gaussianize
from cellink.at.acat import acat_test

In [42]:
DATA = Path(cl.__file__).parent.parent.parent / "docs/tutorials/data"
GENODATA = DATA / "eqtl_cat_genotypes"

gpc_path = GENODATA / "pcdir/OneK1K.noGP.filtered.pruned.eigenvec"
adata_path = DATA / "onk1k_cellxgene_donor_mapped_cd4_naive.h5ad.gz"
gdata_path = GENODATA / "OneK1K.noGP_chr22.vcz"
gene_ann_file = GENODATA / "gene_counts_Ensembl_105_phenotype_metadata.tsv.gz"

In [43]:
n_gpcs = 20
n_epcs = 15
batch_e_pcs_n_top_genes = 2000
chrom = 22
cis_window = 100_000
cell_type = "CD4 Naive"
pb_gex_key = f"PB_{cell_type}"  # pseudobulk expression in dd.G.obsm[key_added]
original_donor_col = "donor_id"
min_percent_donors_expressed = 0.1
celltype_key = "predicted.celltype.l2"
do_debug = False

## Prepare data 

In [44]:
adata = ad.read_h5ad(adata_path)
gdata = cl.io.read_sgkit_zarr(gdata_path)

gene_ann = pd.read_csv(gene_ann_file, sep="\t").set_index("gene_id")
gene_ann = gene_ann[["chromosome", "gene_start", "gene_end", "strand", "gene_name"]].drop_duplicates()
genes_to_keep = list(set(gene_ann.index).intersection(adata.var.index))

genes_to_keep = list(set(gene_ann.index).intersection(adata.var.index))
print(f"Number of genes missing in annotations: {len(adata.var.index) - len(genes_to_keep)}")
adata = adata[:, genes_to_keep]

adata.var = pd.concat([adata.var, gene_ann.loc[adata.var.index]], axis=1).rename(
    columns={
        "gene_start": GAnn.start,
        "gene_end": GAnn.end,
        "chromosome": GAnn.chrom,
        "strand": GAnn.strand,
    }
)
adata.obs[DAnn.donor] = adata.obs[original_donor_col]
adata

Number of genes missing in annotations: 3


AnnData object with n_obs × n_vars = 1248980 × 36466
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'donor_id', 'pool_number', 'predicted.celltype.l2', 'predicted.celltype.l2.score', 'age', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'is_primary_data', 'suspension_type', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type', 'chrom', 'start', 'end', 'strand', 'gene_name'
    uns: 'cell_type_ontology_term_id_colors', 'citation', 'default_embedding', 'schema_refere

In [45]:
dd = cl.DonorData(G=gdata, C=adata).copy()  # copy to avoid view warnings
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │       DonorData(981 x n_donors=136,776, n_cells_per_donor=[333-3,511], donor_id = 'donor_id')    │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 × 136,776 │ AnnData object with n_obs × n_vars = 1,248,980 ×   │
│                                                    │ 36,466                                             │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     obs: 'orig.ident', 'nCount_RNA',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'pool_number', 'predicted.celltype.l2',            │
│ 'TYPED', 'TYPED_ONLY', 'contig', 'id', 'id_mask',  │ 'predicted.celltype.l2.score', 'age',              │
│ 'length', 'quality'                                │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     varm: 'filter'                                 │     var: 'vst.mean', 'vst.variance',               │
│                                                    │ 'vst.variance.expected',                           │
│                                                    │ 'vst.variance.standardized', 'vst.variable',       │
│                                                    │ 'feature_is_filtered', 'feature_name',             │
│                                                    │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type', 'chrom',         │
│                                                    │ 'start', 'end', 'strand', 'gene_name'              │
│                                                    │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title'      │
│                                                    │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'                     │
│                                                    │     varm: 'PCs'                         

### Preprocessing Single-Cell Data
We normalize and log-transform the expression data, compute highly variable genes, and perform PCA to extract expression principal components (ePCs). 

In [46]:
sc.pp.normalize_total(dd.C)
sc.pp.log1p(dd.C)
sc.pp.normalize_total(dd.C)

# are the expression pcs computed by pseudobulking across all cell types?
mdata = sc.get.aggregate(dd.C, by=DAnn.donor, func="mean")
mdata.X = mdata.layers.pop("mean")

sc.pp.highly_variable_genes(mdata, n_top_genes=batch_e_pcs_n_top_genes)
sc.tl.pca(mdata, n_comps=n_epcs)

dd.G.obsm["ePCs"] = mdata[dd.G.obs_names].obsm["X_pca"]

In [47]:
dd = dd[..., dd.C.obs[celltype_key] == cell_type, :].copy()
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │         DonorData(981 x n_donors=136,776, n_cells_per_donor=[6-927], donor_id = 'donor_id')      │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 × 136,776 │ AnnData object with n_obs × n_vars = 259,012 ×     │
│                                                    │ 36,466                                             │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     obs: 'orig.ident', 'nCount_RNA',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'pool_number', 'predicted.celltype.l2',            │
│ 'TYPED', 'TYPED_ONLY', 'contig', 'id', 'id_mask',  │ 'predicted.celltype.l2.score', 'age',              │
│ 'length', 'quality'                                │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     obsm: 'ePCs'                                   │     var: 'vst.mean', 'vst.variance',               │
│                                                    │ 'vst.variance.expected',                           │
│                                                    │ 'vst.variance.standardized', 'vst.variable',       │
│                                                    │ 'feature_is_filtered', 'feature_name',             │
│                                                    │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type', 'chrom',         │
│                                                    │ 'start', 'end', 'strand', 'gene_name'              │
│     varm: 'filter'                                 │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title',     │
│                                                    │ 'log1p'                                            │
│                                                    │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'          

In [48]:
gc.collect()

1914

In [49]:
dd.aggregate(key_added=pb_gex_key, sync_var=True, verbose=True)
dd.aggregate(obs=["sex", "age"], func="first", add_to_obs=True)
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │         DonorData(981 x n_donors=136,776, n_cells_per_donor=[6-927], donor_id = 'donor_id')      │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 × 136,776 │ AnnData object with n_obs × n_vars = 259,012 ×     │
│                                                    │ 36,466                                             │
│     obs: 'sex', 'age'                              │     obs: 'orig.ident', 'nCount_RNA',               │
│                                                    │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│                                                    │ 'pool_number', 'predicted.celltype.l2',            │
│                                                    │ 'predicted.celltype.l2.score', 'age',              │
│                                                    │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     var: 'vst.mean', 'vst.variance',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'vst.variance.expected',                           │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'vst.variance.standardized', 'vst.variable',       │
│ 'TYPED', 'TYPED_ONLY', 'contig', 'id', 'id_mask',  │ 'feature_is_filtered', 'feature_name',             │
│ 'length', 'quality'                                │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type', 'chrom',         │
│                                                    │ 'start', 'end', 'strand', 'gene_name'              │
│     obsm: 'ePCs', 'PB_CD4 Naive'                   │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title',     │
│                                                    │ 'log1p'                                            │
│     varm: 'filter'                                 │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'          

In [50]:
gpcs = pd.read_csv(gpc_path, sep=r"\s+", index_col=1, header=None).drop(columns=0)
dd.G.obsm["gPCs"] = gpcs.loc[dd.G.obs_names].iloc[:, :n_gpcs]

In [51]:
print(f"{pb_gex_key} shape:", dd.G.obsm[pb_gex_key].shape)
print("dd.shape:", dd.shape)

keep_genes = ((dd.G.obsm[pb_gex_key] > 0).mean(axis=0) >= min_percent_donors_expressed).values
dd = dd[..., keep_genes]
print("after filtering")
print(f"{pb_gex_key} shape:", dd.G.obsm[pb_gex_key].shape)
print("dd.shape:", dd.shape)

PB_CD4 Naive shape: (981, 36466)
dd.shape: (981, 136776, 259012, 36466)
after filtering
PB_CD4 Naive shape: (981, 15768)
dd.shape: (981, 136776, 259012, 15768)


In [52]:
# alternative to dd[:, dd.G.var.chrom == str(chrom), :, dd.C.var.chrom == str(chrom)]
dd = dd.sel(G_var=dd.G.var.chrom == str(chrom), C_var=dd.C.var.chrom == str(chrom)).copy()
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │         DonorData(981 x n_donors=136,776, n_cells_per_donor=[6-927], donor_id = 'donor_id')      │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 × 136,776 │ AnnData object with n_obs × n_vars = 259,012 × 413 │
│     obs: 'sex', 'age'                              │     obs: 'orig.ident', 'nCount_RNA',               │
│                                                    │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│                                                    │ 'pool_number', 'predicted.celltype.l2',            │
│                                                    │ 'predicted.celltype.l2.score', 'age',              │
│                                                    │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     var: 'vst.mean', 'vst.variance',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'vst.variance.expected',                           │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'vst.variance.standardized', 'vst.variable',       │
│ 'TYPED', 'TYPED_ONLY', 'contig', 'id', 'id_mask',  │ 'feature_is_filtered', 'feature_name',             │
│ 'length', 'quality'                                │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type', 'chrom',         │
│                                                    │ 'start', 'end', 'strand', 'gene_name'              │
│     obsm: 'ePCs', 'PB_CD4 Naive', 'gPCs'           │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title',     │
│                                                    │ 'log1p'                                            │
│     varm: 'filter'                                 │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'                     │
│                                                    │     varm: 'PCs'                         

### Adding variant annotations to `dd`
We use VEP (Variant Effect Predictor) annotations to add functional information to the variants (as explained [here](./explore_annotations.ipynb)). These annotations are aggregated and stored in the `variant_annotation` attribute of the genotype data.

In [53]:
vep_annotation_file = DATA / "variant_annotation/variants_vep_annotated.txt"

In [54]:
cl.tl.add_vep_annos_to_gdata(vep_anno_file=vep_annotation_file, gdata=dd.G, dummy_consequence=True)
dd.G.uns["variant_annotation_vep"]

,,,Consequence_3_prime_UTR_variant,Consequence_5_prime_UTR_variant,Consequence_NMD_transcript_variant,Consequence_coding_sequence_variant,Consequence_downstream_gene_variant,Consequence_frameshift_variant,Consequence_inframe_deletion,Consequence_inframe_insertion,Consequence_intergenic_variant,Consequence_intron_variant,...,Existing_variation,gnomADe_ASJ_AF,gnomADe_AF,CADD_PHRED,gnomADe_EAS_AF,PHENO,STRAND,CANONICAL,gnomADe_FIN_AF,TSSDistance
snp_id,gene_id,transcript_id,,,,,,,,,,,,,,,,,,,,,
22_16388891_G_A,ENSG00000230643,ENST00000447704,0,0,0,0,1,0,0,0,0,0,...,-,NaN,NaN,NaN,NaN,-,-1.0,YES,NaN,NaN
22_16388968_C_T,ENSG00000230643,ENST00000447704,0,0,0,0,1,0,0,0,0,0,...,-,NaN,NaN,NaN,NaN,-,-1.0,YES,NaN,NaN
22_16389525_A_G,ENSG00000230643,ENST00000447704,0,0,0,0,0,0,0,0,0,0,...,-,NaN,NaN,NaN,NaN,-,-1.0,YES,NaN,NaN
22_16390411_G_A,ENSG00000230643,ENST00000447704,0,0,0,0,0,0,0,0,0,0,...,-,NaN,NaN,NaN,NaN,-,-1.0,YES,NaN,809.0
22_16391555_G_C,ENSG00000230643,ENST00000447704,0,0,0,0,0,0,0,0,0,0,...,-,NaN,NaN,NaN,NaN,-,-1.0,YES,NaN,1953.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22_50796327_CCA_C,ENSG00000100239,ENST00000395741,0,0,0,0,0,0,0,0,0,1,...,-,NaN,NaN,NaN,NaN,-,1.0,YES,NaN,NaN
22_50798021_A_G,ENSG00000100239,ENST00000395741,0,0,0,0,0,0,0,0,0,1,...,-,NaN,NaN,2.263,NaN,-,1.0,YES,NaN,NaN
22_50798635_T_C,ENSG00000100239,ENST00000395741,0,0,0,0,0,0,0,0,0,1,...,-,NaN,NaN,NaN,NaN,-,1.0,YES,NaN,NaN


In [55]:
cl.tl.aggregate_annotations_for_varm(
    dd.G, "variant_annotation_vep", agg_type="first", return_data=True
)  # TODO change agg type

,gene_id,transcript_id,Consequence_3_prime_UTR_variant,Consequence_5_prime_UTR_variant,Consequence_NMD_transcript_variant,Consequence_coding_sequence_variant,Consequence_downstream_gene_variant,Consequence_frameshift_variant,Consequence_inframe_deletion,Consequence_inframe_insertion,...,Existing_variation,gnomADe_ASJ_AF,gnomADe_AF,CADD_PHRED,gnomADe_EAS_AF,PHENO,STRAND,CANONICAL,gnomADe_FIN_AF,TSSDistance
snp_id,,,,,,,,,,,,,,,,,,,,,
22_16388891_G_A,ENSG00000230643,ENST00000447704,0,0,0,0,1,0,0,0,...,-,NaN,NaN,NaN,NaN,-,-1.0,YES,NaN,NaN
22_16388968_C_T,ENSG00000230643,ENST00000447704,0,0,0,0,1,0,0,0,...,-,NaN,NaN,NaN,NaN,-,-1.0,YES,NaN,NaN
22_16389525_A_G,ENSG00000230643,ENST00000447704,0,0,0,0,0,0,0,0,...,-,NaN,NaN,NaN,NaN,-,-1.0,YES,NaN,NaN
22_16390411_G_A,ENSG00000230643,ENST00000447704,0,0,0,0,0,0,0,0,...,-,NaN,NaN,NaN,NaN,-,-1.0,YES,NaN,809.0
22_16391555_G_C,ENSG00000230643,ENST00000447704,0,0,0,0,0,0,0,0,...,-,NaN,NaN,NaN,NaN,-,-1.0,YES,NaN,1953.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22_50796327_CCA_C,ENSG00000100239,ENST00000395741,0,0,0,0,0,0,0,0,...,-,NaN,NaN,NaN,NaN,-,1.0,YES,NaN,NaN
22_50798021_A_G,ENSG00000100239,ENST00000395741,0,0,0,0,0,0,0,0,...,-,NaN,NaN,2.263,NaN,-,1.0,YES,NaN,NaN
22_50798635_T_C,ENSG00000100239,ENST00000395741,0,0,0,0,0,0,0,0,...,-,NaN,NaN,NaN,NaN,-,1.0,YES,NaN,NaN


In [56]:
dd.G.varm["variant_annotation"].columns

Index(['gene_id', 'transcript_id', 'Consequence_3_prime_UTR_variant',
       'Consequence_5_prime_UTR_variant', 'Consequence_NMD_transcript_variant',
       'Consequence_coding_sequence_variant',
       'Consequence_downstream_gene_variant', 'Consequence_frameshift_variant',
       'Consequence_inframe_deletion', 'Consequence_inframe_insertion',
       'Consequence_intergenic_variant', 'Consequence_intron_variant',
       'Consequence_mature_miRNA_variant', 'Consequence_missense_variant',
       'Consequence_non_coding_transcript_exon_variant',
       'Consequence_non_coding_transcript_variant',
       'Consequence_protein_altering_variant',
       'Consequence_splice_acceptor_variant',
       'Consequence_splice_donor_5th_base_variant',
       'Consequence_splice_donor_region_variant',
       'Consequence_splice_donor_variant',
       'Consequence_splice_polypyrimidine_tract_variant',
       'Consequence_splice_region_variant', 'Consequence_start_lost',
       'Consequence_stop_gained

In [57]:
dd.G.uns["variant_annotation_vep"]["CADD_RAW"].describe()

count    32966.000000
mean         0.122248
std          0.542875
min         -1.705125
25%         -0.115726
50%          0.023902
75%          0.205452
max          9.936896
Name: CADD_RAW, dtype: float64

## Run the burden test

Burden tests aggregate the effects of rare variants within a gene or region to test for association with a phenotype (cell type specific expression). We use different weighting schemes, including:
- **CADD_RAW**: Raw CADD scores.
- **maf_beta**: Beta weighting based on minor allele frequency (MAF).
- **tss_distance**: Distance to the transcription start site (TSS).
- **tss_distance_exp**: Exponential weighting based on TSS distance.



In [58]:
burden_agg_fct = "sum"
run_lrt = True
annotation_cols = ["CADD_RAW", "maf_beta", "tss_distance", "tss_distance_exp"]

rare_maf_threshold = 0.05

### Filtering for Rare Variants
We filter variants with a minor allele frequency (MAF) below a specified threshold (e.g., 0.01) to focus on rare variants.

In [59]:
dd = dd.sel(G_var=dd.G.var.maf < rare_maf_threshold).copy()
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │          DonorData(981 x n_donors=39,454, n_cells_per_donor=[6-927], donor_id = 'donor_id')      │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 × 39,454  │ AnnData object with n_obs × n_vars = 259,012 × 413 │
│     obs: 'sex', 'age'                              │     obs: 'orig.ident', 'nCount_RNA',               │
│                                                    │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│                                                    │ 'pool_number', 'predicted.celltype.l2',            │
│                                                    │ 'predicted.celltype.l2.score', 'age',              │
│                                                    │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     var: 'vst.mean', 'vst.variance',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'vst.variance.expected',                           │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'vst.variance.standardized', 'vst.variable',       │
│ 'TYPED', 'TYPED_ONLY', 'contig', 'id', 'id_mask',  │ 'feature_is_filtered', 'feature_name',             │
│ 'length', 'quality'                                │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type', 'chrom',         │
│                                                    │ 'start', 'end', 'strand', 'gene_name'              │
│     uns: 'variant_annotation_vep'                  │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title',     │
│                                                    │ 'log1p'                                            │
│     obsm: 'ePCs', 'PB_CD4 Naive', 'gPCs'           │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'                     │
│     varm: 'filter', 'variant_annotation'           │     varm: 'PCs'                         

### Custom MAF Weights

We add custom MAF weights commonly used in burden tests, such as `Beta(MAF, 1, 25)`. These weights prioritize rarer variants in the analysis. TSS distance weight as used in the SAIGE-QTL paper are added manually for each gene

In [60]:
dd.G.varm["variant_annotation"]["maf_beta"] = beta_weighting(dd.G.var["maf"])

### Run burden tests using each annotation individually for weighting


In [61]:
# This specifies covariates/fixed effects
F = np.concatenate(
    [
        np.ones((dd.shape[0], 1)),
        dd.G.obs[["sex"]].values - 1,
        dd.G.obs[["age"]].values,
        dd.G.obsm["gPCs"].values,
        dd.G.obsm["ePCs"],
    ],
    axis=1,
)
F[:, 2:] = column_normalize(F[:, 2:])

In [62]:
results = []
if isinstance(dd.G.X, da.Array | ad._core.views.DaskArrayView):
    if dd.G.is_view:
        dd._G = dd._G.copy()
    dd.G.X = dd.G.X.compute()

if do_debug:
    warnings.filterwarnings("ignore", category=RuntimeWarning)

for gene, row in tqdm(dd.C.var.iterrows(), total=dd.shape[3]):
    Y = gaussianize(dd.G.obsm[pb_gex_key][[gene]].values + 1e-5 * np.random.randn(dd.shape[0], 1))

    start = max(0, row.start - cis_window)
    end = row.end + cis_window
    _G = dd.G[:, (dd.G.var.pos < end)]
    _G = _G[:, (_G.var.pos > start)]
    _G = _G[:, (_G.X.std(0) != 0)]
    _G = _G.copy()

    # TODO make strand aware
    _G.varm["variant_annotation"]["tss_distance"] = np.abs(row.start - _G.var["pos"])
    _G.varm["variant_annotation"]["tss_distance_exp"] = np.exp(-1e-5 * _G.varm["variant_annotation"]["tss_distance"])

    rdf = run_burden_test(
        _G, Y, F, gene, annotation_cols=annotation_cols, burden_agg_fct=burden_agg_fct, run_lrt=run_lrt
    )
    results.append(rdf)

rdf = pd.concat(results)
rdf

 35%|███▍      | 143/413 [00:05<00:10, 26.53it/s]/data/nasif12/home_if12/hoev/git/sc-genetics/src/cellink/at/gwas.py:57: RuntimeWarning: divide by zero encountered in divide
  n = 1.0 / (GG - np.einsum("ij,ij->j", FG, A0iFG))
/data/nasif12/home_if12/hoev/git/sc-genetics/src/cellink/at/gwas.py:58: RuntimeWarning: invalid value encountered in multiply
  M = -n * A0iFG
/data/nasif12/home_if12/hoev/git/sc-genetics/src/cellink/at/gwas.py:62: RuntimeWarning: invalid value encountered in multiply
  self.beta_g += n[:, None] * GY
 46%|████▋     | 192/413 [00:07<00:08, 26.15it/s]/data/nasif12/home_if12/hoev/git/sc-genetics/src/cellink/at/gwas.py:57: RuntimeWarning: divide by zero encountered in divide
  n = 1.0 / (GG - np.einsum("ij,ij->j", FG, A0iFG))
/data/nasif12/home_if12/hoev/git/sc-genetics/src/cellink/at/gwas.py:58: RuntimeWarning: invalid value encountered in multiply
  M = -n * A0iFG
/data/nasif12/home_if12/hoev/git/sc-genetics/src/cellink/at/gwas.py:62: RuntimeWarning: invalid value e

,burden_gene,egene,weight_col,burden_agg_fct,pv,beta,betaste,lrt
0,ENSG00000221963,ENSG00000221963,CADD_RAW,sum,NaN,NaN,NaN,NaN
1,ENSG00000221963,ENSG00000221963,maf_beta,sum,0.302384,1.289730e-04,1.250546e-04,1.063649
2,ENSG00000221963,ENSG00000221963,tss_distance,sum,0.601209,2.502974e-08,4.788891e-08,0.273176
3,ENSG00000221963,ENSG00000221963,tss_distance_exp,sum,0.269160,2.309883e-03,2.090394e-03,1.221022
0,ENSG00000272669,ENSG00000272669,CADD_RAW,sum,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3,ENSG00000235111,ENSG00000235111,tss_distance_exp,sum,0.853691,-3.785719e-04,2.052866e-03,0.034008
0,ENSG00000185022,ENSG00000185022,CADD_RAW,sum,NaN,NaN,NaN,NaN
1,ENSG00000185022,ENSG00000185022,maf_beta,sum,0.379449,2.329100e-04,2.649982e-04,0.772486
2,ENSG00000185022,ENSG00000185022,tss_distance,sum,0.529773,7.027998e-08,1.118479e-07,0.394828


### Combine p-values per gene across anntoations using ACAT

We use the ACAT test to combine p-values across different annotations for each gene. This approach provides a single p-value per gene, accounting for multiple annotations.

In [63]:
combined = rdf.dropna(subset=["pv"]).groupby("egene")["pv"].agg(lambda x: acat_test(x.values)).reset_index()
combined.sort_values("pv")

,egene,pv
84,ENSG00000100219,2.331468e-14
290,ENSG00000212939,1.176343e-10
75,ENSG00000100154,5.907117e-05
198,ENSG00000167077,1.569339e-04
186,ENSG00000133460,1.576775e-04
...,...,...
22,ENSG00000099940,9.787134e-01
28,ENSG00000099958,9.865699e-01
195,ENSG00000167037,9.871363e-01
350,ENSG00000261202,9.934300e-01


## SKAT tests
At the moment only default weighting with weights = Beta(MAF, 1, 25) is supported

In [ ]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.WARNING)  # Suppress INFO and DEBUG esp. from SKAT Test

In [65]:
results = []

for gene, row in tqdm(dd.C.var.iterrows(), total=dd.shape[3]):
    Y = gaussianize(dd.G.obsm[pb_gex_key][[gene]].values + 1e-5 * np.random.randn(dd.shape[0], 1))

    start = max(0, row.start - cis_window)
    end = row.end + cis_window
    _G = dd.G[:, (dd.G.var.pos < end)]
    _G = _G[:, (_G.var.pos > start)]
    _G = _G[:, (_G.X.std(0) != 0)]

    rdict = run_skat_test(_G, Y, F, gene)
    results.append(rdict)

rdf = pd.DataFrame(results)
rdf

  0%|          | 0/413 [00:00<?, ?it/s]

 35%|███▍      | 144/413 [00:10<00:16, 16.47it/s]/opt/modules/i12g/anaconda/envs/eh_scgenetics_v1/lib/python3.10/site-packages/chiscore/_davies.py:134: RuntimeWarning: invalid value encountered in scalar divide
  s1 = c1[2] / c1[1] ** (3 / 2)
/opt/modules/i12g/anaconda/envs/eh_scgenetics_v1/lib/python3.10/site-packages/chiscore/_davies.py:135: RuntimeWarning: invalid value encountered in scalar divide
  s2 = c1[3] / c1[1] ** 2
/opt/modules/i12g/anaconda/envs/eh_scgenetics_v1/lib/python3.10/site-packages/chiscore/_davies.py:113: RuntimeWarning: invalid value encountered in divide
  Q_Norm = (Q_all - param["muQ"]) / param["sigmaQ"]
 47%|████▋     | 194/413 [00:13<00:14, 15.30it/s]/opt/modules/i12g/anaconda/envs/eh_scgenetics_v1/lib/python3.10/site-packages/chiscore/_davies.py:134: RuntimeWarning: invalid value encountered in scalar divide
  s1 = c1[2] / c1[1] ** (3 / 2)
/opt/modules/i12g/anaconda/envs/eh_scgenetics_v1/lib/python3.10/site-packages/chiscore/_davies.py:135: RuntimeWarning: 

,burden_gene,egene,weight_col,pv
0,ENSG00000221963,ENSG00000221963,maf_beta,0.836209
1,ENSG00000272669,ENSG00000272669,maf_beta,0.982043
2,ENSG00000100416,ENSG00000100416,maf_beta,0.500403
3,ENSG00000079974,ENSG00000079974,maf_beta,0.041599
4,ENSG00000183741,ENSG00000183741,maf_beta,0.169456
...,...,...,...,...
408,ENSG00000213923,ENSG00000213923,maf_beta,0.177634
409,ENSG00000236499,ENSG00000236499,maf_beta,0.418554
410,ENSG00000183963,ENSG00000183963,maf_beta,0.502256
411,ENSG00000235111,ENSG00000235111,maf_beta,0.116578


In [66]:
rdf

,burden_gene,egene,weight_col,pv
0,ENSG00000221963,ENSG00000221963,maf_beta,0.836209
1,ENSG00000272669,ENSG00000272669,maf_beta,0.982043
2,ENSG00000100416,ENSG00000100416,maf_beta,0.500403
3,ENSG00000079974,ENSG00000079974,maf_beta,0.041599
4,ENSG00000183741,ENSG00000183741,maf_beta,0.169456
...,...,...,...,...
408,ENSG00000213923,ENSG00000213923,maf_beta,0.177634
409,ENSG00000236499,ENSG00000236499,maf_beta,0.418554
410,ENSG00000183963,ENSG00000183963,maf_beta,0.502256
411,ENSG00000235111,ENSG00000235111,maf_beta,0.116578
